Leaving this information after collecting it from docs and multiple discussions etc,

- **src (torch.Tensor)** – the sequence to the encoder (required). shape: (𝑁,𝑆,𝐸).
- **tgt (torch.Tensor)** – the sequence to the decoder (required). shape: (𝑁,𝑇,𝐸).
- **src_mask (torch.Tensor, optional)** – the additive mask for the src sequence (optional). shape: (𝑆,𝑆).
- **tgt_mask (torch.Tensor, optional)** – the additive mask for the tgt sequence (optional). shape: (𝑇,𝑇).
- **memory_mask (torch.Tensor, optional)** – the additive mask for the encoder output (optional). shape: (𝑇,𝑆).
- **src_key_padding_mask (torch.Tensor, optional)** – the ByteTensor mask for src keys per batch (optional). shape: (𝑁,𝑆)
- **tgt_key_padding_mask (torch.Tensor, optional)** – the ByteTensor mask for tgt keys per batch (optional). shape: (𝑁,𝑇).
- **memory_key_padding_mask (torch.Tensor, optional)** – the ByteTensor mask for memory keys per batch (optional). shape” (𝑁,𝑆).

- **output (torch.Tensor)** – The output sequence, shape: (𝑁,𝑇,𝐸).

- **Note ([src/tgt/memory]_mask** should be filled with) float(‘-inf’) for the masked positions and float(0.0) else.

- **These masks ensure that predictions for position i depend only on the unmasked positions j and are applied identically for each sequence in a batch.** 

- **[src/tgt/memory]_key_padding_mask should be a ByteTensor where False values are positions that should be masked with float(‘-inf’) and True values will be unchanged. This mask ensures that no information will be taken from position i if it is masked, and has a separate mask for each sequence in a batch.**


# # answered correctly is not padded

In [ ]:
# import some typical packages and fix random seed

import numpy as np
import pandas as pd
import pickle
import gc
import time
import lightgbm as lgbm
import itertools


from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score, auc, roc_curve
from contextlib import contextmanager
from pathlib import Path
from time import time
from pathlib import Path
from collections import namedtuple

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
gc.enable()

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math

from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
from torch.nn.modules.dropout import Dropout
from torch.nn.modules.activation import MultiheadAttention
from torch.nn.modules.normalization import LayerNorm

TRAIN_DTYPES = {
    # 'row_id': np.uint32,
    'timestamp': np.uint64,
    'user_id': np.uint32,
    'content_id': np.uint16,
    'content_type_id': np.uint8,
    'task_container_id': np.uint16,
    'user_answer': np.int8,
    'answered_correctly': np.int8,
    'prior_question_elapsed_time': np.float32,
    'prior_question_had_explanation': 'boolean'
}

DATA_DIR = Path('../input/riiid-test-answer-prediction')
TRAIN_PATH = DATA_DIR / 'train.csv'
QUESTIONS_PATH = DATA_DIR / 'questions.csv'
LECTURES_PATH = DATA_DIR / 'lectures.csv'

'''
Masking in the encoder is required to make sure any padding doesn't contribute to the self-attention mechanism. 
In Pytorch, this is done by passing src_key_padding_mask to the transformer. 
For the example, this looks like [False, False, False, False, False, False, False, True, True, True] 
where the True positions should be masked. The output of the encoder is called memory.
''';

'''
S is the source sequence length, N is the batch size, E is the feature number.
src: (S, N, E) = (source sequence len, batch size, feature number)
tgt: (T, N, E) = (target sequence len, batch size, feature number)
src_mask: (S, S) = the additive mask for the src sequence
tgt_mask: (T, T) = (target sequence len, target sequence len)
src_key_padding_mask: (N, S) = (batch size, source sequence len)
tgt_key_padding_mask: (N, T) = (batch size, target sequence len)
memory_key_padding_mask: (N, S) = (batch size, source sequence len)

key_padding_mask controls how which batch items are allowed to attend to certain key positions. 
This is most commonly used to avoid attending to padding elements. 

attn_mask controls how query positions are allowed to attend to key positions. 
This is useful for doing left-to-right (causal) attention, where we enforce that query positions are 
only allowed to attend to keys to their left.
''';

from sklearn.metrics import accuracy_score

class MaskedCrossEntropyLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.loss_ = nn.CrossEntropyLoss()

    def forward(self, input, target, mask=None):
        # Flatten to (BS*seq_len)
        input_ = input.reshape(-1)
        target_ = target.reshape(-1)
        if mask is not None:
            mask_ = mask.reshape(-1)
            input_ = input_[mask_ == False]
            target_ = target_[mask_ == False]
        return self.loss_(input_, target_)

    
def roc_auc_compute_fn(y_targets, y_preds):
    try:
        from sklearn.metrics import roc_auc_score
    except ImportError:
        raise RuntimeError("This contrib module requires sklearn to be installed.")
    

    y_true = y_targets.detach().cpu()
    y_pred = y_preds.detach().cpu()
    
    return roc_auc_score(y_true, y_pred)


def accuracy_score_compute_fn(y_targets, y_preds):
    try:
        from sklearn.metrics import roc_auc_score
    except ImportError:
        raise RuntimeError("This contrib module requires sklearn to be installed.")
    
    # ON CPU
    y_true = y_targets.detach().cpu()
    y_pred = y_preds.detach().cpu()
#     y_true = y_targets.cpu()
#     y_pred = y_preds.cpu()
    return accuracy_score(y_true, y_pred)

torch.cuda.is_available()

# Preparing the Dataset

In [ ]:
%%time

CV_NO = 2
LAST_N = 100

PAD = 0
SOS_TOKEN = 2
PAD_ANS = 3
TIMELINESS_PAD = 2
CNT = 0
LAG_PAD = 0

df_questions = pd.read_csv(QUESTIONS_PATH)
part_ids_map = dict(zip(df_questions.question_id, df_questions.part))


# answered correctly is not padded
with open("../input/saint-inference-user-id-as-key-pkls/d_train_2_saint_inference_with_lag.pkl/d_train_2_saint_inference_with_lag.pkl", "rb") as fid:
    d_train = pickle.load(fid)


# with open("../input/saint-inference-user-id-as-key-pkls/cache/d_valid_2_saint_inference.pkl", "rb") as fid:
#     d_valid = pickle.load(fid)

In [ ]:
print(d_train[115])

In [ ]:
# Source: https://pytorch.org/tutorials/beginner/transformer_tutorial
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


# Source: https://pytorch.org/tutorials/beginner/transformer_tutorial
class PositionalEncoding(nn.Module):
    '''
    A piece of information added to each word about its position in the sentence in Transformers.
    Even Indices Sine; Oddd Indices Cosine;
    Had replaced math.log with np.log;
    '''
    def __init__(self, d_model, dropout=0.1, max_len=LAST_N):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


# The below implements the optimizer that SAINT uses.
class NoamOpt(object):
    r'''Noam Optim wrapper that implements rate.
    Three settings of the lrate hyperparameters.
    opts = [NoamOpt(512, 1, 4000, None), NoamOpt(512, 1, 8000, None), NoamOpt(256, 1, 4000, None)]
    plt.plot(np.arange(1, 20000), [[opt.rate(i) for opt in opts] for i in range(1, 20000)])
    plt.legend(["512:4000", "512:8000", "256:4000"])
    '''
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        r"Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * (self.model_size ** (-0.5) * min(step ** (-0.5), step * self.warmup ** (-1.5)))


def grab_optimizer(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000, torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))


class raw_conf:

    mtype = "SAINT"
    backbone = "transformer" 

    pad_mode = "random" # i.e. pad with data from another user
    seq_len = 100
    embedding_dim = 256 # embed_dim must be divisible by num_heads
    exercices_id_size = 13782
    exercices_part_size = 7
    response_size = 2 
    elapsed_time_size = 300 
    lag_time_size = 720 # It was 1440 in SAINT paper.
    explanation_size = 2
    position_encoding_enabled = True

    # Model
    nhead = 8
    num_encoder_layers = 4
    num_decoder_layers = 4
    dim_feedforward = 2048
    dropout = 0.1
    activation = None

In [ ]:
class SAINT(nn.Module):

    def __init__(self,
                 d_model: int = 64,
                 nhead: int = 8, 
                 num_encoder_layers: int = 2, num_decoder_layers: int = 2,
                 dim_feedforward: int = 64, dropout: float = 0.1, activation: str = 'relu',
                 device="cpu"
        ):
        
        super(SAINT, self).__init__()

        encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, activation)
        encoder_norm = LayerNorm(d_model)
        self.encoder = TransformerEncoder(encoder_layer, num_encoder_layers, encoder_norm)
        
        decoder_layer = TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout, activation)
        decoder_norm = LayerNorm(d_model)
        self.decoder = TransformerDecoder(decoder_layer, num_decoder_layers, decoder_norm)

        self._reset_parameters() # inits the weights and all by default

        self.d_model = d_model
        self.nhead = nhead
        self.SEQ_LEN = 100
        self.device = device
        
        self.POS_START_TOKEN = 0
        self.CORRECTNESS_START_TOKEN = 2
        self.ELAPSED_START_TOKEN = 0
        self.LAG_START_TOKEN = 1
        
        # NB position embeddings weights are tied wrt encoder and decoder.
        
        # ---------------#        
        ## ENCODER SIDE ##
        # ---------------#
        
        self.exercise_embeddings = nn.Embedding(num_embeddings=13524+1, embedding_dim=d_model, padding_idx=0)# Exercise
        self.pos_embedding = PositionalEncoding(d_model=d_model, max_len=LAST_N) # nn.Embedding(self.SEQ_LEN+1, d_model, padding_idx=0) # Position
        self.part_embeddings = nn.Embedding(num_embeddings=7+1, embedding_dim=d_model) # Part
        # self.timeliness_embeddings = nn.Embedding(num_embeddings=2+1, embedding_dim=d_model, padding_idx=2)
        
        # ---------------#
        ## DECODER SIDE ##
        # ---------------#
        
        # Position Embeddings -> shared b/w them
        self.correctness_embedding = nn.Embedding(num_embeddings=2+1+1, embedding_dim=d_model, padding_idx=3)# Correctness
        self.prior_question_elapsed_time = nn.Embedding(num_embeddings=301, embedding_dim=d_model)# Elapsed Time
        self.lag_time = nn.Embedding(num_embeddings=301+1+1, embedding_dim=d_model, padding_idx=0) # Lag Time
        
        self.ffn = nn.Linear(d_model, 2)
        
    def _reset_parameters(self):
        r"""Initiate parameters in the transformer model."""

        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def _generate_square_subsequent_mask(self, sz):
        r"""
        For training.
        The masked positions are filled with float('-inf'). 
        Unmasked positions are filled with float(0.0).
        """
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def _generate_mem_mask(self, tgt_sz=LAST_N, src_sz=LAST_N):
        mask = (torch.triu(torch.ones(src_sz, tgt_sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self,
                lag=None,
                content_id=None, part_id=None, prior_question_elapsed_time=None, # timeliness=None,
                answered_correctly=None, src_key_padding_mask=None, test=False,
        ):
        r"""Take in and process masked source/target sequences.

        Args:
            src: the sequence to the encoder (required).
            tgt: the sequence to the decoder (required).
            src_mask: the additive mask for the src sequence (optional).
            tgt_mask: the additive mask for the tgt sequence (optional).
            memory_mask: the additive mask for the encoder output (optional).
            src_key_padding_mask: the ByteTensor mask for src keys per batch (optional).
            tgt_key_padding_mask: the ByteTensor mask for tgt keys per batch (optional).
            memory_key_padding_mask: the ByteTensor mask for memory keys per batch (optional).

        Shape:
            - src: :math:`(S, N, E)`.
            - tgt: :math:`(T, N, E)`.
            - src_mask: :math:`(S, S)`.
            - tgt_mask: :math:`(T, T)`.
            - memory_mask: :math:`(T, S)`.
            - src_key_padding_mask: :math:`(N, S)`.
            - tgt_key_padding_mask: :math:`(N, T)`.
            - memory_key_padding_mask: :math:`(N, S)`.

            Note: 
            
            [src/tgt/memory]_mask ensures that position "i" is allowed to attend the unmasked
            positions. If a ByteTensor is provided, the non-zero positions are not allowed to attend
            while the zero positions will be unchanged. If a BoolTensor is provided, positions with ``True``
            are not allowed to attend while ``False`` values will be unchanged. If a FloatTensor
            is provided, it will be added to the attention weight.
            
            [src/tgt/memory]_key_padding_mask provides specified elements in the key to be ignored by
            the attention. If a ByteTensor is provided, the non-zero positions will be ignored while the zero
            positions will be unchanged. If a BoolTensor is provided, the positions with the
            value of ``True`` will be ignored while the position with the value of ``False`` will be unchanged.

            - output: :math:`(T, N, E)`.

            Note: Due to the multi-head attention architecture in the transformer model,
            the output sequence length of a transformer is same as the input sequence
            (i.e. target) length of the decode.

            where S is the source sequence length, T is the target sequence length, N is the
            batch size, E is the feature number

        Examples:
            >>> output = transformer_model(src, tgt, src_mask=src_mask, tgt_mask=tgt_mask)
        """
        
        device = self.device
        # ENCODER SIDE
        src_mask = self._generate_square_subsequent_mask(self.SEQ_LEN).to(device)

        # using nn.Embedding Only as Positional Embedding
        # embedded_src = self.exercise_embeddings(content_id) + \
        # self.pos_embedding(torch.arange(0, content_id.shape[1]).to(self.device).unsqueeze(0).repeat(content_id.shape[0], 1)) + \
        # self.part_embeddings(part_id) + self.prior_question_elapsed_time(prior_question_elapsed_time)
        
        # BATCH_SIZE, MAX_SENT_LEN, EMBEDDING_DIM (N, S, E)
        embedded_src = self.exercise_embeddings(content_id) + \
        self.part_embeddings(part_id) + self.prior_question_elapsed_time(prior_question_elapsed_time) # + self.timeliness_embeddings(timeliness)
        
        # BATCH_SIZE, SEQ_LEN, EMBEDDING_DIM (N, S, E)
        embedded_src = self.pos_embedding(embedded_src.permute(1,0,2)) # pos_enc(src.permute(1, 0, 2))
        
        embedded_src = embedded_src * np.sqrt(self.d_model)
        
        # (S, N, E)
        _memory = self.encoder(src=embedded_src, mask=src_mask, src_key_padding_mask=src_key_padding_mask)
        del embedded_src
        
        # DECODER SIDE
        
        tgt_mask = self._generate_square_subsequent_mask(self.SEQ_LEN).to(device)
        memory_mask = self._generate_square_subsequent_mask(self.SEQ_LEN).to(device)
        
        tgt_key_padding_mask = src_key_padding_mask.clone() # <TO-DO>; here T and S are same length
        memory_key_padding_mask = src_key_padding_mask.clone()
        
        # Correctness
        x_correctness = answered_correctly.long()
        # x_correctness =  torch.roll(answered_correctly.long(), shifts=(0, 1, 0), dims=(0, 1, 0)) # Shift right the sequence
        # x_correctness[:,0] = self.CORRECTNESS_START_TOKEN
        
        # Position handled via embeddings.
        
        # Elapsed Time
        x_elapsed = prior_question_elapsed_time
        # x_elapsed =  torch.roll(prior_question_elapsed_time, shifts=(0, 1, 0), dims=(0, 1, 0))# Shift right the sequence
        # x_elapsed[:,0] = self.ELAPSED_START_TOKEN

        # Lag Time
        x_lag = lag
        # x_lag =  torch.roll(lag, shifts=(0, 1, 0), dims=(0, 1, 0))# Shift right the sequence
        # x_lag[:,0] = self.LAG_START_TOKEN
        
        # BATCH_SIZE, MAX_SENT_LEN, EMBEDDING_DIM (N, S, E)
        embedded_tgt = self.correctness_embedding(x_correctness) + self.prior_question_elapsed_time(x_elapsed)
        embedded_tgt = self.pos_embedding(embedded_tgt.permute(1,0,2)) # pos_enc(src.permute(1, 0, 2))

        # (S, N, E)
        # embedded_tgt = embedded_tgt.transpose(0, 1)
        embedded_tgt = embedded_tgt * np.sqrt(self.d_model)
        
        output = self.decoder(
            tgt=embedded_tgt, memory=_memory,
            tgt_mask=None, memory_mask=memory_mask, tgt_key_padding_mask=content_id==0,
            memory_key_padding_mask=memory_key_padding_mask
        )
        
        output = output.transpose(1, 0) # (N, S, E)
        output = self.ffn(output)
        
        if test == True:
            # To mask future tokens, you should use src_mask, tgt_mask, memory_mask
            # output = output.contiguous().view(-1, 2)
            predicted_probs = torch.softmax(output[:,-1,:], dim=1)[:,1]
            return predicted_probs
        return output

In [ ]:
DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

print(DEVICE)

model = SAINT(
    d_model = 512,
    nhead = 4,
    num_encoder_layers = 4, num_decoder_layers = 4,
    dim_feedforward = 256, dropout = 0.1, activation = 'relu', device=DEVICE
)

model.to(DEVICE);
model.eval();

BATCH_SIZE = 64 # *64
lr = 1e-4 # learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
%%time
model.eval();
model.load_state_dict(torch.load("../input/saint-model-weights/Archive 2/saint_2_21_hidden_256_d_model_512_layers_4_with_lag.pt", map_location=DEVICE));
# torch.save(model.state_dict(), f'saint_testing.pt')

'''
    #########################################
    #thanks to @tksasagi for her tweet#######
    #########################################
    print('|￣￣￣￣￣￣￣￣|')
    print('|    VALIDATION    |') 
    print('|     Fold %i prediction cv AUC: %.5f  |' %(i,cv_auc))
    print('| ＿＿＿_＿＿＿＿|') 
    print(' (\__/) ||') 
    print(' (•ㅅ•) || ')
    print(' / 　 づ')
''';

# INFERENCE ATTEMPT - 1

In [ ]:
%%time

import os, copy
import multiprocessing
from multiprocessing import Process, Queue, Pool
import time

TARGET = 'answered_correctly'
STATE = {}
PART_IDS_MAP = part_ids_map
PRIOR_QUESTION_ELAPSED_MEAN_FILLNA = 26000
MAX_SEQ = 100

print("TILL TRAINING: 554169193",)


# with open("../input/saint-inference-user-id-as-key-pkls/cache/d_train_2_saint_inference.pkl", "rb") as fid:
#     STATE = pickle.load(fid)

STATE = d_train

In [ ]:
%%time

gc.collect()

import torch
import riiideducation
from time import time
from contextlib import contextmanager

env = riiideducation.make_env()
iter_test = env.iter_test()

previous_test_df = None

class Colors:
    """Defining Color Codes to color the text displayed on terminal.
    """

    blue = "\033[94m"
    green = "\033[92m"
    yellow = "\033[93m"
    red = "\033[91m"
    end = "\033[0m"


def color(string: str, color: Colors = Colors.yellow) -> str:
    return f"{color}{string}{Colors.end}"


@contextmanager
def timer(label: str) -> None:
    """compute the time the code block takes to run.
    """
    start = time()  # Setup - __enter__
    print(color(f"{label}: Start at {start}"))
    try:
        yield  # yield to body of `with` statement
    finally:  # Teardown - __exit__
        end = time()
        print(color(f"{label}: End at {end} ({end - start} elapsed secs)", color=Colors.red))


gc.collect()

In [ ]:
%%time

with open("../input/gen-all-users-past-content-ids/last_seen_timestamp.pkl", "rb") as fid:
    last_seen_timestamp = pickle.load(fid)

In [ ]:
last_seen_timestamp[115]

In [ ]:
last_seen_timestamp[115.0]

In [ ]:
print(d_train[554169193])

In [ ]:
len(d_train[554169193]["content_id"]), len(d_train[554169193]["answered_correctly"]), len(d_train[554169193]['part_id'])

In [ ]:
# print(STATE[275030867]) # BEFORE ANYTHING

In [ ]:
%%time

gc.collect();
model.eval();

MAX_SEQ = 100
NEW_IDS = []


for idx, (test_df, sample_prediction_df) in enumerate(iter_test):
    _content_id, _part_id, _prior_question_elapsed_time, _src_key_padding_mask, _timeliness, _answered_correctly, _lag = [], [], [], [], [], [], []
    
    if previous_test_df is not None:

        previous_test_df[TARGET] = eval(test_df["prior_group_answers_correct"].iloc[0])
        previous_test_df = previous_test_df[previous_test_df['content_type_id'] == 0].reset_index(drop=True)
        previous_test_df = previous_test_df[['user_id', TARGET]].to_numpy()
        
        for (user_id, target) in previous_test_df:
            # old_user, new_user issue will come here as well but no NEW_USER will be unseen at this stage.
            _len = STATE[user_id]['len']
            if _len == 1 and len(set(STATE[user_id][TARGET])) == 1:
                STATE[user_id][TARGET] = []
                STATE[user_id][TARGET].append(target)
            else:
                if _len >= 100:
#                     if user_id in [555691277]:
#                         print("doing for", user_id, target)
#                         print("BEFORE UPDATION", STATE[user_id])
                    STATE[user_id][TARGET] = STATE[user_id][TARGET][-99:] + [target]
#                     if user_id in [555691277]:
#                         print("AFTER UPDATION", STATE[user_id])
                else:
                    STATE[user_id][TARGET].append(target)

    
    previous_test_df = test_df.copy()
    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop=True)
    test_df["part_id"] = test_df["content_id"].map(PART_IDS_MAP)
    test_df['prior_question_elapsed_time'] = test_df.prior_question_elapsed_time.fillna(PRIOR_QUESTION_ELAPSED_MEAN_FILLNA) // 1000
    
    # test_df["timeliness"] = test_df[["part_id", "prior_question_elapsed_time"]].apply(lambda x: get_timeliness(x["part_id"], x["prior_question_elapsed_time"]), axis=1)*1
    
    # for lag and content_id, we have added a +2 to them; # one for padding and other for SOS
    test_df["content_id"] += 2
    # test_df["lag"] = test_df["lag"].clip(lower=1000, upper=300_000) // 1000
    # test_df["lag"] = test_df["lag"].fillna(0)
    # test_df["lag"] += 2

    curr = test_df[["user_id", "content_id", "part_id", "prior_question_elapsed_time", "timestamp"]].to_numpy()
    
    for (user_id, content_id, part_id, prior_question_elapsed_time, timestamp) in curr:
        # user_id = user_id
        if user_id in STATE:
            # great news for us actually, so, let's add the values and restrict them
            # using padded sequences of MAX_SEQ as 100

            UPDATE_IDX = STATE[user_id]['len']
            lag = (np.clip(timestamp - last_seen_timestamp[user_id]['timestamp'], 1_000, 300_000) // 1000) + 2
            last_seen_timestamp[user_id]['timestamp'] = timestamp
            
#             # creating the batch we need here by ouselves
#             _content_id.append(STATE[user_id]['content_id'])
#             _part_id.append(STATE[user_id]['part_id'])
#             _prior_question_elapsed_time.append(STATE[user_id]['prior_question_elapsed_time'])
#             # _timeliness.append(STATE[user_id]['timeliness'])
#             _lag.append(STATE[user_id]['lag'])
#             _src_key_padding_mask.append(STATE[user_id]['src_key_padding_mask'])

#             # padding it on FLY here itself
#             _answered_correctly.append(STATE[user_id]['answered_correctly'] + [3]*(100 - len(STATE[user_id]['answered_correctly'])))
            
            if UPDATE_IDX >= MAX_SEQ:
                STATE[user_id]['content_id'] = STATE[user_id]['content_id'][-99:] + [content_id]
                STATE[user_id]['part_id'] = STATE[user_id]['part_id'][-99:] + [part_id]
                STATE[user_id]['prior_question_elapsed_time'] = STATE[user_id]['prior_question_elapsed_time'][-99:] + [prior_question_elapsed_time]
                STATE[user_id]['lag'] = STATE[user_id]['lag'][-99:] + [lag]
                # STATE[user_id]['timeliness'] = STATE[user_id]['timeliness'][-99:] + [timeliness]
                STATE[user_id]['len'] = MAX_SEQ
            else:
                STATE[user_id]['content_id'][UPDATE_IDX] = content_id
                STATE[user_id]['part_id'][UPDATE_IDX] = part_id
                STATE[user_id]['prior_question_elapsed_time'][UPDATE_IDX] = prior_question_elapsed_time
                # STATE[user_id]['timeliness'][UPDATE_IDX] = timeliness
                STATE[user_id]['lag'][UPDATE_IDX] = lag
                STATE[user_id]['src_key_padding_mask'][UPDATE_IDX] = False

            # creating the batch we need here by ouselves
            _content_id.append(STATE[user_id]['content_id'])
            _part_id.append(STATE[user_id]['part_id'])
            _prior_question_elapsed_time.append(STATE[user_id]['prior_question_elapsed_time'])
            # _timeliness.append(STATE[user_id]['timeliness'])
            _lag.append(STATE[user_id]['lag'])
            _src_key_padding_mask.append(STATE[user_id]['src_key_padding_mask'])

            # padding it on FLY here itself
            _answered_correctly.append(STATE[user_id]['answered_correctly'] + [3]*(100 - len(STATE[user_id]['answered_correctly'])))

            # point to next updation place except if at-max, then it's MAX_SEQ always.
            STATE[user_id]['len'] = min(MAX_SEQ, STATE[user_id]['len'] + 1)

        else:
            NEW_IDS.append(user_id)
            # A NEW USER
            
#             _content_id.append([0] * (MAX_SEQ))
#             _part_id.append([0] * (MAX_SEQ))
#             _prior_question_elapsed_time.append([0] * (MAX_SEQ))
#             _src_key_padding_mask.append([True] * (MAX_SEQ))
#             _lag.append([0] * (MAX_SEQ))
#             _answered_correctly.append([3.] * MAX_SEQ)
            
            STATE[user_id] = {
                'content_id': [content_id] + [0] * (MAX_SEQ-1),
                'part_id': [part_id] + [0] * (MAX_SEQ-1),
                'prior_question_elapsed_time': [prior_question_elapsed_time] + [0] * (MAX_SEQ-1),
                'src_key_padding_mask': [False] + [True] * (MAX_SEQ-1),
                # "timeliness": [timeliness] + [TIMELINESS_PAD] * (MAX_SEQ-1),
                'answered_correctly': [3.] * MAX_SEQ, # empty basically.
                "lag": [0] + [0] * (MAX_SEQ-1), # don't start WITH 0 ???????
                'len': 1,
            }
            
            # add it as well
            _content_id.append(STATE[user_id]['content_id']) # should have start token ?
            _part_id.append(STATE[user_id]['part_id'])
            _prior_question_elapsed_time.append(STATE[user_id]['prior_question_elapsed_time'])
            _src_key_padding_mask.append(STATE[user_id]['src_key_padding_mask'])
            _lag.append(STATE[user_id]['lag'])
            _answered_correctly.append(STATE[user_id]['answered_correctly'])
            
            # reset it
            STATE[user_id]['answered_correctly'] = []
            last_seen_timestamp[user_id] = {'timestamp' : 0}
            # print("NEW ENTRY FOR", user_id, STATE[user_id])
    
    with torch.no_grad():
        _content_id = torch.Tensor(_content_id).to(DEVICE).long()
        _part_id = torch.Tensor(_part_id).to(DEVICE).long()
        _prior_question_elapsed_time = torch.Tensor(_prior_question_elapsed_time).to(DEVICE).long()
        _src_key_padding_mask = torch.Tensor(_src_key_padding_mask).to(DEVICE).bool()
        _answered_correctly = torch.Tensor(_answered_correctly).to(DEVICE)
        _lag = torch.Tensor(_lag).to(DEVICE).long()

        test_df['answered_correctly'] = model(
                content_id=_content_id, part_id=_part_id, 
                prior_question_elapsed_time=_prior_question_elapsed_time, 
                src_key_padding_mask=_src_key_padding_mask,
                lag=_lag,
                answered_correctly=_answered_correctly,
                test=True,
        ).cpu()

        env.predict(test_df.loc[:, ['row_id', 'answered_correctly']])

In [ ]:
print(STATE[275030867])

In [ ]:
_content_id

In [ ]:
_part_id

In [ ]:
mask = _content_id == 0
mask.masked_fill(mask, -np.inf)

In [ ]:
test_df[["row_id", "answered_correctly"]].values